In [20]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Define the directory containing the datasets
data_dir = "../../data/"
dataset_path = os.path.join(data_dir, "analyzed/catHarmQA/combined_catqa.csv")

In [21]:
data = pd.read_csv(dataset_path)

## 1. Original Question -> Response through all Models -> Labels from Llama Guard

In [22]:
# Count the number of safe and unsafe responses of each model for original questions
data.groupby("model")["original_response_safety"].value_counts(normalize=True).unstack()

original_response_safety,safe,unsafe
model,,
llama2,0.320000,0.680000
llama3,0.180000,0.820000
llama31,0.216364,0.783636
mistral,0.176364,0.823636


### 1.1 Cross verification

In [23]:
# Define a helper function to filter the safety responses
def filter_safety_response(label):
    return label.strip().split()[0].lower()

# Load individual model safety data
llama2_org_ques_safety_df = pd.read_csv(os.path.join(data_dir, "safety/catHarmQA/response/catqa_llama2_Question_safety.csv"))
llama3_org_ques_safety_df = pd.read_csv(os.path.join(data_dir, "safety/catHarmQA/response/catqa_llama3_Question_safety.csv"))
llama31_org_ques_safety_df = pd.read_csv(os.path.join(data_dir, "safety/catHarmQA/response/catqa_llama31_Question_safety.csv"))
mistral_org_ques_safety_df = pd.read_csv(os.path.join(data_dir, "safety/catHarmQA/response/catqa_mistral_Question_safety.csv"))

# Apply filtering and calculate normalized proportions for each model
pd.DataFrame({
            "llama2": llama2_org_ques_safety_df['Question_llama2_safety'].apply(filter_safety_response).value_counts(normalize=True),
            "llama3": llama3_org_ques_safety_df['Question_llama3_safety'].apply(filter_safety_response).value_counts(normalize=True),
            "llama31": llama31_org_ques_safety_df['Question_llama31_safety'].apply(filter_safety_response).value_counts(normalize=True),
            "mistral": mistral_org_ques_safety_df['Question_mistral_safety'].apply(filter_safety_response).value_counts(normalize=True)
        }).fillna(0).T

,unsafe,safe
llama2,0.680000,0.320000
llama3,0.820000,0.180000
llama31,0.783636,0.216364
mistral,0.823636,0.176364


## 2. Original Question -> Get Unsafe Original Question using LLama-Guard -> Response through all Models -> Labels from Llama Guard

In [24]:
data.original_question_safety.value_counts(normalize=True)

original_question_safety
unsafe    0.932727
safe      0.067273
Name: proportion, dtype: float64

In [25]:
# filtered unsafe original question according to llama guard
data[data["original_question_safety"] == "unsafe"].groupby("model")["original_response_safety"].value_counts(normalize=True).unstack()

original_response_safety,safe,unsafe
model,,
llama2,0.280702,0.719298
llama3,0.130604,0.869396
llama31,0.169591,0.830409
mistral,0.126706,0.873294


## 3. Original Question -> Do Perturbation(char/word/sntnc) -> Check Safety of Perturbed Question -> Take only unsafe perturbed question -> Response through all Models -> Labels from Llama Guard

### 3.1. Safe percetnage of char, word, sentnc

In [26]:
# Safety percentages for perturbed questions
data[data["perturbed_question_safety"] == "unsafe"].groupby("perturbation_level").perturbed_response_safety.value_counts(normalize=True).unstack()

perturbed_response_safety,safe,unsafe
perturbation_level,,
char,0.181836,0.818164
sntnc,0.223295,0.776705
word,0.184400,0.815600


In [27]:
# Safety percentages for perturbed questions
data[data["perturbed_question_safety"] == "unsafe"].groupby(
    "perturbation_level"
).perturbed_response_pre_safety.value_counts(normalize=True).unstack()

perturbed_response_pre_safety,safe,unsafe
perturbation_level,,
char,0.661362,0.338638
sntnc,0.632670,0.367330
word,0.647100,0.352900


### 3.2. re-verification

In [28]:
for level in ["char","word", "sntnc"]:
    dd = data[data.perturbation_level == level]
    dd = dd[dd.perturbed_question_safety == "unsafe"]
    dd = dd.perturbed_response_safety.value_counts(normalize=True)*100
    print(f"{level} : {dd.safe:.2f}")

char : 18.18
word : 18.44
sntnc : 22.33


In [29]:
for level in ["char","word", "sntnc"]:
    dd = data[data.perturbation_level == level]
    dd = dd[dd.perturbed_question_safety == "unsafe"]
    dd = dd.perturbed_response_pre_safety.value_counts(normalize=True)*100
    print(f"{level} : {dd.safe:.2f}")

char : 66.14
word : 64.71
sntnc : 63.27


### 3.3. Unsafe

In [30]:
data[data["perturbed_question_safety"] == "unsafe"].groupby(["model" , "perturbation_level"])["perturbed_response_safety"].value_counts(normalize=True).unstack() * 100

perturbed_response_safety        safe     unsafe
model   perturbation_level                      
llama2  char                28.177702  71.822298
        sntnc               35.113636  64.886364
        word                29.150157  70.849843
llama3  char                12.448323  87.551677
        sntnc               18.295455  81.704545
        word                12.973084  87.026916
llama31 char                16.444649  83.555351
        sntnc               19.659091  80.340909
        word                16.074600  83.925400
mistral char                15.663757  84.336243
        sntnc               16.250000  83.750000
        word                15.562235  84.437765

In [31]:
data[data["perturbed_question_safety"] == "unsafe"].groupby(["model" , "perturbation_level"])["perturbed_response_pre_safety"].value_counts(normalize=True).unstack() * 100

perturbed_response_pre_safety       safe     unsafe
model   perturbation_level                         
llama2  char                   90.957412   9.042588
        sntnc                  83.750000  16.250000
        word                   89.124197  10.875803
llama3  char                   44.300807  55.699193
        sntnc                  49.545455  50.454545
        word                   46.010384  53.989616
llama31 char                   68.344380  31.655620
        sntnc                  65.000000  35.000000
        word                   64.366717  35.633283
mistral char                   60.942319  39.057681
        sntnc                  54.772727  45.227273
        word                   59.338707  40.661293

In [ ]:
# Set display options
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)  # Do not wrap columns
pd.set_option("display.max_colwidth", None)  # Do not truncate column values

data[data["perturbed_question_safety"] == "unsafe"].groupby(
    ["model", "perturbation_level", "category"]
)["d"].value_counts(normalize=True).unstack() * 100

perturbed_response_safety                                   safe     unsafe
model   perturbation_level category                                        
llama2  char               Adult Content               22.237106  77.762894
                           Child Abuse                 16.862474  83.137526
                           Economic Harm               27.983252  72.016748
                           Fraud/Deception             24.602626  75.397374
                           Hate/Harass/Violence        25.409836  74.590164
                           Illegal Activity            16.221766  83.778234
                           Malware Viruses             21.716738  78.283262
                           Physical Harm               39.670556  60.329444
                           Political Campaigning       45.617978  54.382022
                           Privacy Violation Activity  34.062500  65.937500
                           Tailored Financial Advice   37.867940  62.132060
        sntnc              Adult Content               31.958763  68.041237
                           Child Abuse                 19.767442  80.232558
                           Economic Harm               35.802469  64.197531
                           Fraud/Deception             41.758242  58.241758
                           Hate/Harass/Violence        33.333333  66.666667
                           Illegal Activity            25.806452  74.193548
                           Malware Viruses             19.696970  80.303030
                           Physical Harm               41.860465  58.139535
                           Political Campaigning       60.563380  39.436620
                           Privacy Violation Activity  45.312500  54.687500
                           Tailored Financial Advice   34.375000  65.625000
        word               Adult Content               21.851600  78.148400
                           Child Abuse                 18.596237  81.403763
                           Economic Harm               30.479705  69.520295
                           Fraud/Deception             26.742532  73.257468
                           Hate/Harass/Violence        27.800830  72.199170
                           Illegal Activity            16.516932  83.483068
                           Malware Viruses             22.465753  77.534247
                           Physical Harm               36.715997  63.284003
                           Political Campaigning       44.426624  55.573376
                           Privacy Violation Activity  39.601386  60.398614
                           Tailored Financial Advice   39.490969  60.509031
llama3  char               Adult Content                9.511052  90.488948
                           Child Abuse                  6.565308  93.434692
                           Economic Harm               12.002791  87.997209
                           Fraud/Deception             11.472011  88.527989
                           Hate/Harass/Violence         7.581967  92.418033
                           Illegal Activity             9.308693  90.691307
                           Malware Viruses             11.759657  88.240343
                           Physical Harm               19.903912  80.096088
                           Political Campaigning       12.808989  87.191011
                           Privacy Violation Activity  14.218750  85.781250
                           Tailored Financial Advice   23.468576  76.531424
        sntnc              Adult Content               20.618557  79.381443
                           Child Abuse                  9.302326  90.697674
                           Economic Harm               20.987654  79.012346
                           Fraud/Deception             18.681319  81.318681
                           Hate/Harass/Violence        18.518519  81.481481
                           Illegal Activity            11.827957  88.172043
                           Malware Viruses      

In [33]:
# Set display options
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)  # Do not wrap columns
pd.set_option("display.max_colwidth", None)  # Do not truncate column values

data[data["perturbed_question_safety"] == "unsafe"].groupby(
    ["model", "perturbation_level", "category"]
)["perturbed_response_pre_safety"].value_counts(normalize=True).unstack() * 100

perturbed_response_pre_safety                               safe     unsafe
model   perturbation_level category                                        
llama2  char               Adult Content               97.320831   2.679169
                           Child Abuse                 92.328956   7.671044
                           Economic Harm               92.114445   7.885555
                           Fraud/Deception             91.430546   8.569454
                           Hate/Harass/Violence        98.292350   1.707650
                           Illegal Activity            89.869952  10.130048
                           Malware Viruses             77.424893  22.575107
                           Physical Harm               91.146191   8.853809
                           Political Campaigning       90.337079   9.662921
                           Privacy Violation Activity  87.421875  12.578125
                           Tailored Financial Advice   89.260143  10.739857
        sntnc              Adult Content               93.814433   6.185567
                           Child Abuse                 86.046512  13.953488
                           Economic Harm               80.246914  19.753086
                           Fraud/Deception             83.516484  16.483516
                           Hate/Harass/Violence        98.765432   1.234568
                           Illegal Activity            83.870968  16.129032
                           Malware Viruses             57.575758  42.424242
                           Physical Harm               84.883721  15.116279
                           Political Campaigning       91.549296   8.450704
                           Privacy Violation Activity  81.250000  18.750000
                           Tailored Financial Advice   70.312500  29.687500
        word               Adult Content               94.962560   5.037440
                           Child Abuse                 92.185239   7.814761
                           Economic Harm               88.782288  11.217712
                           Fraud/Deception             90.967283   9.032717
                           Hate/Harass/Violence        97.164592   2.835408
                           Illegal Activity            88.666206  11.333794
                           Malware Viruses             75.433790  24.566210
                           Physical Harm               90.063425   9.936575
                           Political Campaigning       87.249399  12.750601
                           Privacy Violation Activity  83.795494  16.204506
                           Tailored Financial Advice   86.042693  13.957307
llama3  char               Adult Content               43.536504  56.463496
                           Child Abuse                 34.830684  65.169316
                           Economic Harm               49.965108  50.034892
                           Fraud/Deception             44.160332  55.839668
                           Hate/Harass/Violence        39.207650  60.792350
                           Illegal Activity            38.261465  61.738535
                           Malware Viruses             41.802575  58.197425
                           Physical Harm               47.975292  52.024708
                           Political Campaigning       40.449438  59.550562
                           Privacy Violation Activity  50.390625  49.609375
                           Tailored Financial Advice   58.711217  41.288783
        sntnc              Adult Content               51.546392  48.453608
                           Child Abuse                 32.558140  67.441860
                           Economic Harm               74.074074  25.925926
                           Fraud/Deception             49.450549  50.549451
                           Hate/Harass/Violence        49.382716  50.617284
                           Illegal Activity            39.784946  60.215054
                           Malware Viruses      

In [34]:
data[data["perturbed_question_safety"] == "unsafe"].groupby(
    ["perturbation_level" , "category"]
)["perturbed_response_safety"].value_counts(normalize=True).unstack().T * 100

perturbation_level                 char                            \
category                  Adult Content Child Abuse Economic Harm   
perturbed_response_safety                                           
safe                          14.651708   12.629578      17.00977   
unsafe                        85.348292   87.370422      82.99023   

perturbation_level                                              \
category                  Fraud/Deception Hate/Harass/Violence   
perturbed_response_safety                                        
safe                            15.739461            13.592896   
unsafe                          84.260539            86.407104   

perturbation_level                                                        \
category                  Illegal Activity Malware Viruses Physical Harm   
perturbed_response_safety                                                  
safe                             11.601643       15.472103     27.350721   
unsafe                           88.398357       84.527897     72.649279   

perturbation_level                                                          \
category                  Political Campaigning Privacy Violation Activity   
perturbed_response_safety                                                    
safe                                  22.565543                  21.054688   
unsafe                                77.434457                  78.945312   

perturbation_level                                          sntnc              \
category                  Tailored Financial Advice Adult Content Child Abuse   
perturbed_response_safety                                                       
safe                                      30.230708     20.618557   13.372093   
unsafe                                    69.769292     79.381443   86.627907   

perturbation_level                                                            \
category                  Economic Harm Fraud/Deception Hate/Harass/Violence   
perturbed_response_safety                                                      
safe                          23.148148       22.527473             21.91358   
unsafe                        76.851852       77.472527             78.08642   

perturbation_level                                                        \
category                  Illegal Activity Malware Viruses Physical Harm   
perturbed_response_safety                                                  
safe                             14.516129       14.015152     30.813953   
unsafe                           85.483871       85.984848     69.186047   

perturbation_level                                                          \
category                  Political Campaigning Privacy Violation Activity   
perturbed_response_safety                                                    
safe                                  28.873239                   30.46875   
unsafe                                71.126761                   69.53125   

perturbation_level                                           word              \
category                  Tailored Financial Advice Adult Content Child Abuse   
perturbed_response_safety                                                       
safe                                      29.296875     15.180395   14.652677   
unsafe                                    70.703125     84.819605   85.347323   

perturbation_level                                                            \
category                  Economic Harm Fraud/Deception Hate/Harass/Violence   
perturbed_response_safety                                                      
safe                          17.398524       15.647226            14.298064   
unsafe                        82.601476       84.352774            85.701936   

perturbation_level                                                        \
category                  Illegal Activity Malware Viruses Physical Harm   
perturbed_res

In [35]:
data[data["perturbed_question_safety"] == "unsafe"].groupby(
    ["perturbation_level", "category"]
)["perturbed_response_pre_safety"].value_counts(normalize=True).unstack().T * 100

perturbation_level                     char                            \
category                      Adult Content Child Abuse Economic Harm   
perturbed_response_pre_safety                                           
safe                              70.562626   64.581894     68.684578   
unsafe                            29.437374   35.418106     31.315422   

perturbation_level                                                  \
category                      Fraud/Deception Hate/Harass/Violence   
perturbed_response_pre_safety                                        
safe                                64.754665            67.127732   
unsafe                              35.245335            32.872268   

perturbation_level                                                            \
category                      Illegal Activity Malware Viruses Physical Harm   
perturbed_response_pre_safety                                                  
safe                                 58.675565       56.909871     70.041181   
unsafe                               41.324435       43.090129     29.958819   

perturbation_level                                   \
category                      Political Campaigning   
perturbed_response_pre_safety                         
safe                                       61.70412   
unsafe                                     38.29588   

perturbation_level                                        \
category                      Privacy Violation Activity   
perturbed_response_pre_safety                              
safe                                           68.554688   
unsafe                                         31.445312   

perturbation_level                                              sntnc  \
category                      Tailored Financial Advice Adult Content   
perturbed_response_pre_safety                                           
safe                                           75.13922     65.979381   
unsafe                                         24.86078     34.020619   

perturbation_level                                                       \
category                      Child Abuse Economic Harm Fraud/Deception   
perturbed_response_pre_safety                                             
safe                            59.302326     68.518519       61.263736   
unsafe                          40.697674     31.481481       38.736264   

perturbation_level                                                   \
category                      Hate/Harass/Violence Illegal Activity   
perturbed_response_pre_safety                                         
safe                                      68.82716        53.763441   
unsafe                                    31.17284        46.236559   

perturbation_level                                           \
category                      Malware Viruses Physical Harm   
perturbed_response_pre_safety                                 
safe                                49.242424     75.290698   
unsafe                              50.757576     24.709302   

perturbation_level                                   \
category                      Political Campaigning   
perturbed_response_pre_safety                         
safe                                      62.676056   
unsafe                                    37.323944   

perturbation_level                                        \
category                      Privacy Violation Activity   
perturbed_response_pre_safety                              
safe                                                62.5   
unsafe                                              37.5   

perturbation_level                                               word  \
category                      Tailored Financial Advice Adult Content   
perturbed_response_pre_safety                                           
safe                                            67.1875     67.903336   
unsafe                

In [36]:
data.category.value_counts(normalize=True) * 100

category
Adult Content                 9.090909
Child Abuse                   9.090909
Economic Harm                 9.090909
Fraud/Deception               9.090909
Hate/Harass/Violence          9.090909
Illegal Activity              9.090909
Malware Viruses               9.090909
Physical Harm                 9.090909
Political Campaigning         9.090909
Privacy Violation Activity    9.090909
Tailored Financial Advice     9.090909
Name: proportion, dtype: float64